In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import VECM
from itertools import combinations
from google.colab import files
import matplotlib.pyplot as plt


# Fetching tickers from Wikipedia pages of S&P500, S&P400, S&P600
def get_tickers():
    urls = {
        "S&P 500": "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
        "S&P 400": "https://en.wikipedia.org/wiki/List_of_S%26P_400_companies",
        "S&P 600": "https://en.wikipedia.org/wiki/List_of_S%26P_600_companies"
    }

    tickers = set()  # Use a set to avoid duplicates

    for url in urls.values():
        df = pd.read_html(url)[0]

        # Identify the correct ticker column
        ticker_column = [col for col in df.columns if 'Ticker' in col or 'Symbol' in col][0]

        # Replace '.' with '-' to match yfinance format
        tickers.update(df[ticker_column].str.replace(".", "-", regex=False).dropna().tolist())

    return sorted(tickers)  # Return sorted list of unique tickers


def get_log_returns(tickers):
    data = yf.download(tickers, period="12y", interval="1d")["Close"]
    data = data.ffill().bfill()  # Fill missing values using previous and next available data
    log_returns = np.log(data / data.shift(1)).fillna(0)
    return log_returns


def get_data_from_csv():
    # Load the CSV file
    df = pd.read_csv('/content/1500stockdata.csv', parse_dates=['Date'], index_col='Date', dayfirst=True)

    # Calculate log returns for all tickers at once
    log_returns = np.log(df / df.shift(1))

    # Fill NaN values (from the first row)
    log_returns = log_returns.fillna(0)

    return log_returns

def find_coint_pairs(tickers, log_returns):

    sig_level = 0.0005
    # Compute correlation matrix
    corr_matrix = log_returns.corr()

    # Find highly correlated pairs (e.g., correlation > 0.9)
    high_coint_pairs = []
    threshold = 0.9
    upper_threshold = 0.99

    for ticker1, ticker2 in combinations(tickers, 2):
        if corr_matrix.loc[ticker1, ticker2] > threshold and corr_matrix.loc[ticker1, ticker2] < upper_threshold:  # Only consider pairs that are highly correlated but not perfectly (0.99)
            if log_returns[ticker1].std() == 0 or log_returns[ticker2].std() == 0:
                continue  # Skip pairs where one stock has zero variance

            score, p_value, x = coint(log_returns[ticker1], log_returns[ticker2])

            if p_value < sig_level:  # If there is sufficient evidence to suggest the stocks are cointegrated
              high_coint_pairs.append([ticker1, ticker2])

    return high_coint_pairs


# Fit a VECM to find cointegrating relationships
def run_vecm(data):
    model = VECM(data, k_ar_diff=1, coint_rank=1)  # Rank 1 because we are testing pairs
    vecm_fit = model.fit()
    residuals = vecm_fit.resid  # Extract error correction term
    hedge_ratios = vecm_fit.beta[:, 0]  # First cointegrating vector
    return residuals, hedge_ratios

# Generates trade signals and calculates profit/loss for a specific year
def generate_trade_signals(data, hedge_ratios, year):

    # Filter data for the specified year
    data = data[data.index.year == year]

    # Ensure hedge_ratios is a column vector
    hedge_ratios = np.array(hedge_ratios).reshape(-1, 1)

    # Compute the spread (stationary component)
    spread = data.dot(hedge_ratios)

    # Standardize the spread
    spread_mean = spread.mean()
    spread_std = spread.std()
    z_score = (spread - spread_mean) / spread_std

    # Define trading signals
    long_signal = z_score < -1  # Go long when spread is very negative
    short_signal = z_score > 1   # Go short when spread is very positive
    exit_signal = abs(z_score) < 0.5  # Exit when spread normalizes

    # Construct position dataframe
    positions = pd.DataFrame(index=data.index)
    positions[data.columns[0]] = long_signal.astype(int) - short_signal.astype(int)
    positions[data.columns[1]] = -positions[data.columns[0]]

    # Shift positions to avoid look-ahead bias
    shifted_positions = positions.shift(1).fillna(0)

    # Calculate daily returns based on position and log returns
    daily_returns = (shifted_positions * data).sum(axis=1)

    # Compute cumulative PnL
    pnl = daily_returns.cumsum()

    return positions, pnl

# Computes annualised_returns, sharpe ratio, max drawdown, volatility and calmar ratio
def compute_performance_metrics(cum_pnl):

    # Convert cumulative PnL to daily returns
    daily_returns = cum_pnl.pct_change().dropna()

    # Annualized Return (assuming 252 trading days in a year)
    total_return = cum_pnl.iloc[-1] / cum_pnl.iloc[0] - 1
    num_years = (cum_pnl.index[-1] - cum_pnl.index[0]).days / 365
    annualised_return = (1 + total_return) ** (1 / num_years) - 1

    # Sharpe Ratio (assuming risk-free rate is 0)
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)

    # Maximum Drawdown
    rolling_max = cum_pnl.cummax()
    drawdown = (cum_pnl - rolling_max) / rolling_max
    max_drawdown = drawdown.min()

    # Volatility (Annualized Standard Deviation)
    volatility = daily_returns.std() * np.sqrt(252)

    # Calmar Ratio (Annualized Return / Max Drawdown)
    calmar_ratio = annualised_return / abs(max_drawdown) if max_drawdown != 0 else np.nan

    # Print the metrics
    print(f"Annualised Return: {annualised_return:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Maximum Drawdown: {max_drawdown:.2%}")
    print(f"Volatility: {volatility:.2%}")
    print(f"Calmar Ratio: {calmar_ratio:.2f}")

    return {
        "Annualised Return": annualised_return,
        "Sharpe Ratio": sharpe_ratio,
        "Maximum Drawdown": max_drawdown,
        "Volatility": volatility,
        "Calmar Ratio": calmar_ratio
    }


if __name__ == "__main__":

    tickers = get_tickers()
    log_returns = get_log_returns(tickers)

    # Alternative (much quicker) method to get log returns: using pre-downloaded data
    # Downloaded data from yfinance onto CSV file (link: https://www.dropbox.com/scl/fi/r5776193mjszok9u1sel9/1500stockdata.csv?rlkey=wj0ux6k8ne2q3klv8vw3rhs35&st=mj8wyx14&dl=0)
    # Download the csv then upload when requested
    #uploaded = files.upload()
    #log_returns = get_data_from_csv()
    #log_returns.index = pd.to_datetime(log_returns.index)  # Convert to DateTime
    #log_returns = log_returns.asfreq('B')  # Set frequency to business days
    #log_returns = log_returns.fillna(0)

    log_returns_transpose = log_returns.T # In the format num_stocks x num_dates

    tickers = log_returns.columns.tolist()

    years = sorted(log_returns.index.year.unique())  # Get unique years in dataset
    cum_pnl = pd.Series(dtype=float)  # Start as an empty Series
    last_multiplier = 1  # Start at 1 (representing $1)

    for year in years:
        yearly_pnl = 0  # Initialising yearly_pnl as 0
        print(f"Processing year: {year}")

        # Get previous year's data to find cointegrated pairs
        yearly_data = log_returns[log_returns.index.year == year - 1]

        # If there's no data for the previous year, skip
        if yearly_data.empty:
            continue

        coint_pairs = find_coint_pairs(tickers, yearly_data)
        print(f"Pairs for {year}: {coint_pairs}")

        # If no cointegrated pairs were found, skip this year
        if not coint_pairs:
            continue

        pair_count = 0 # counting no of coint pairs considered, so we can normalise the yearly_pnl later on

        for pair in coint_pairs:
            data = yearly_data[pair]  # Get log return data for the pair

            # Skip if variance is too low
            if np.var(data.values) < 1e-6:
                continue

            pair_count+=1

            residuals, hedge_ratio = run_vecm(data)
            # Use only the correct year's data
            pos, pnl = generate_trade_signals(log_returns[log_returns.index.year == year][pair], hedge_ratio, year)
            yearly_pnl += pnl

        yearly_pnl = yearly_pnl/pair_count + 1
        yearly_pnl *= last_multiplier

        last_multiplier = yearly_pnl.iloc[-1]

        if not yearly_pnl.empty:  # Only concatenate if yearly_pnl is not empty
            cum_pnl = pd.concat([cum_pnl, yearly_pnl])


    # Plot Cumulative PnL
    plt.figure(figsize=(12, 6))
    plt.plot(cum_pnl.index, cum_pnl, label="Cumulative PnL", color='blue')

    # Formatting the plot
    plt.xlabel("Time")
    plt.ylabel("Cumulative PnL")
    plt.title("Strategy Cumulative PnL Over Time")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)  # Add a grid for better readability

    plt.show()

    metrics = compute_performance_metrics(cum_pnl) # Print the metrics
